In [1]:
import numpy as np

In [ ]:
desired_state = 3

ar = np.genfromtxt('../data/A/combined_traces/10uM_idl.csv', delimiter=',')

In [2]:
# Normalization array
ar_N = ar.copy()
ar_N[np.where(ar == 1)] = 0  # photobleached or not idealized
ar_N[np.where(ar >= 2)] = 1  # one of FRET states

# Basal state array
ar_b = ar.copy()
ar_b[np.where(ar != desired_state)] = 0
ar_b[np.where(ar == desired_state)] = 1

pop = ar_b.sum(axis=0) / ar_N.sum(axis=0)

C:\Users\rkiselev\AppData\Local\Temp\ipykernel_29204\2906279062.py:14: RuntimeWarning: invalid value encountered in divide
  pop = ar_b.sum(axis=0) / ar_N.sum(axis=0)
